In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType, StructType, StructField
from pyspark.ml.feature import StringIndexer, VectorAssembler, QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark import SparkContext
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors, VectorUDT
import pandas as pd
import numpy as np
from sklearn import metrics 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
spark = SparkSession.builder.appName("Titanic data ").getOrCreate()


In [3]:
data = spark.read.csv("/FileStore/tables/Titanic1.csv", header="true", inferSchema="true")
display(data)

last,first,gender,age,Pclass,fare,embarked,survived
Braund,Mr. Owen Harris,M,22.0,3,7.25,Southampton,no
Cumings,Mrs. John Bradley (Florence Briggs Thayer),F,38.0,1,71.2833,Cherbourg,yes
Heikkinen,Miss Laina,F,26.0,3,7.925,Southampton,yes
Futrelle,Mrs. Jacques Heath (Lily May Peel),F,35.0,1,53.1,Southampton,yes
Allen,Mr. William Henry,M,35.0,3,8.05,Southampton,no
Moran,Mr. James,M,null,3,8.4583,Queenstown,no
McCarthy,Mr. Timothy J,M,54.0,1,51.8625,Southampton,no
Palsson,Master Gosta Leonard,M,2.0,3,21.075,Southampton,no
Johnson,Mrs. Oscar W (Elisabeth Vilhelmina Berg),F,27.0,3,11.1333,Southampton,yes
Nasser,Mrs. Nicholas (Adele Achem),F,14.0,2,30.0708,Cherbourg,yes


In [4]:
data.count()

Out[36]: 891

In [5]:
data.createOrReplaceTempView("data")

In [6]:
empty_columns =[]
for col_name in data.columns:
    empty_values = data.where(F.col(col_name).isNull()).count()
    if(empty_values > 0):
        empty_columns.append((col_name, empty_values))
print(empty_columns)

[('age', 177)]

In [7]:
round(spark.sql("SELECT AVG(age) FROM data").collect()[0][0])


Out[39]: 30

In [8]:
data = data.fillna(30, subset=['age'])


In [9]:
labels = [StringIndexer(inputCol=column, outputCol=column+"_new").fit(data) for column in ["gender","embarked","survived"]]

pipeline = Pipeline(stages=labels)
data = pipeline.fit(data).transform(data)

data.show()

+-------------+--------------------+------+----+------+-------+-----------+--------+----------+------------+------------+
 last| first|gender| age|Pclass| fare| embarked|survived|gender_new|embarked_new|survived_new|
+-------------+--------------------+------+----+------+-------+-----------+--------+----------+------------+------------+
 Braund| Mr. Owen Harris| M|22.0| 3| 7.25|Southampton| no| 0.0| 0.0| 0.0|
 Cumings|Mrs. John Bradley...| F|38.0| 1|71.2833| Cherbourg| yes| 1.0| 1.0| 1.0|
 Heikkinen| Miss Laina| F|26.0| 3| 7.925|Southampton| yes| 1.0| 0.0| 1.0|
 Futrelle|Mrs. Jacques Heat...| F|35.0| 1| 53.1|Southampton| yes| 1.0| 0.0| 1.0|
 Allen| Mr. William Henry| M|35.0| 3| 8.05|Southampton| no| 0.0| 0.0| 0.0|
 Moran| Mr. James| M|30.0| 3| 8.4583| Queenstown| no| 0.0| 2.0| 0.0|
 McCarthy| Mr. Timothy J| M|54.0| 1|51.8625|Southampton| no| 0.0| 0.0| 0.0|
 Palsson|Master Gosta Leonard| M| 2.0| 3| 21.075|Southampton| no| 0.0| 0.0| 0.0|
 Johnson|Mrs. Oscar W (Eli...| F|27.0| 3|11.1333|Southampton| yes| 1.0| 0.0| 1.0|
 Nasser|Mrs. Nicholas (Ad...| F|14.0| 2|30.0708| Cherbourg| yes| 1.0| 1.0| 1.0|
 Sandstrom| Miss Marguerite Rut| F| 4.0| 3| 16.7|Southampton| yes| 1.0| 0.0| 1.0|
 Bonnell| Miss Elizabeth| F|58.0| 1| 26.55|Southampton| yes| 1.0| 0.0| 1.0|
 Saundercock| Mr. William Henry| M|20.0| 3| 8.05|Southampton| no| 0.0| 0.0| 0.0|
 Andersson| Mr. Anders Johan| M|39.0| 3| 31.275|Southampton| no| 0.0| 0.0| 0.0|
 Vestrom|Miss Hulda Amanda...| F|14.0| 3| 7.8542|Southampton| no| 1.0| 0.0| 0.0|
 Hewlett|Mrs. (Mary D King...| F|55.0| 2| 16.0|Southampton| yes| 1.0| 0.0| 1.0|
 Rice| Master Eugene| M| 2.0| 3| 29.125| Queenstown| no| 0.0| 2.0| 0.0|
 Williams| Mr. Charles Eugene| M|30.0| 2| 13.0|Southampton| yes| 0.0| 0.0| 1.0|
Vander Planke|Mrs. Julius (Emel...| F|31.0| 3| 18.0|Southampton| no| 1.0| 0.0| 0.0|
 Masselmani| Mrs. Fatima| F|30.0| 3| 7.225| Cherbourg| yes| 1.0| 1.0| 1.0|
+-------------+--------------------+------+----+------+-------+-----------+--------+----------+------------+------------+
only showing top 20 rows

In [10]:
data = data.drop('gender','last','first','fare', 'embarked', 'survived')

In [11]:
# Survived with respect to gender
display(data)

age,Pclass,gender_new,embarked_new,survived_new
22.0,3,0.0,0.0,0.0
38.0,1,1.0,1.0,1.0
26.0,3,1.0,0.0,1.0
35.0,1,1.0,0.0,1.0
35.0,3,0.0,0.0,0.0
30.0,3,0.0,2.0,0.0
54.0,1,0.0,0.0,0.0
2.0,3,0.0,0.0,0.0
27.0,3,1.0,0.0,1.0
14.0,2,1.0,1.0,1.0


In [12]:
data.groupBy('survived_new','gender_new').count().show()

+------------+----------+-----+
survived_new|gender_new|count|
+------------+----------+-----+
 1.0| 1.0| 233|
 0.0| 1.0| 81|
 1.0| 0.0| 109|
 0.0| 0.0| 468|
+------------+----------+-----+

In [13]:
#Survive with respect to Pclass 
display(data)

age,Pclass,gender_new,embarked_new,survived_new
22.0,3,0.0,0.0,0.0
38.0,1,1.0,1.0,1.0
26.0,3,1.0,0.0,1.0
35.0,1,1.0,0.0,1.0
35.0,3,0.0,0.0,0.0
30.0,3,0.0,2.0,0.0
54.0,1,0.0,0.0,0.0
2.0,3,0.0,0.0,0.0
27.0,3,1.0,0.0,1.0
14.0,2,1.0,1.0,1.0


In [14]:
data.groupBy('Pclass', 'survived_new','gender_new').count().show()

+------+------------+----------+-----+
Pclass|survived_new|gender_new|count|
+------+------------+----------+-----+
 1| 0.0| 0.0| 77|
 3| 1.0| 1.0| 72|
 3| 0.0| 1.0| 72|
 3| 0.0| 0.0| 300|
 1| 1.0| 0.0| 45|
 3| 1.0| 0.0| 47|
 1| 0.0| 1.0| 3|
 2| 1.0| 1.0| 70|
 2| 0.0| 0.0| 91|
 2| 0.0| 1.0| 6|
 1| 1.0| 1.0| 91|
 2| 1.0| 0.0| 17|
+------+------------+----------+-----+

In [15]:
# Survive with respect to age
display(data)

age,Pclass,gender_new,embarked_new,survived_new
22.0,3,0.0,0.0,0.0
38.0,1,1.0,1.0,1.0
26.0,3,1.0,0.0,1.0
35.0,1,1.0,0.0,1.0
35.0,3,0.0,0.0,0.0
30.0,3,0.0,2.0,0.0
54.0,1,0.0,0.0,0.0
2.0,3,0.0,0.0,0.0
27.0,3,1.0,0.0,1.0
14.0,2,1.0,1.0,1.0


In [16]:
# Survive with respect to embarked 
display(data)

age,Pclass,gender_new,embarked_new,survived_new
22.0,3,0.0,0.0,0.0
38.0,1,1.0,1.0,1.0
26.0,3,1.0,0.0,1.0
35.0,1,1.0,0.0,1.0
35.0,3,0.0,0.0,0.0
30.0,3,0.0,2.0,0.0
54.0,1,0.0,0.0,0.0
2.0,3,0.0,0.0,0.0
27.0,3,1.0,0.0,1.0
14.0,2,1.0,1.0,1.0


In [17]:
data.groupBy('gender_new').count().show()

+----------+-----+
gender_new|count|
+----------+-----+
 0.0| 577|
 1.0| 314|
+----------+-----+

In [18]:
data.groupBy('embarked_new','survived_new').count().show()

+------------+------------+-----+
embarked_new|survived_new|count|
+------------+------------+-----+
 2.0| 0.0| 47|
 1.0| 1.0| 93|
 0.0| 1.0| 219|
 1.0| 0.0| 75|
 2.0| 1.0| 30|
 0.0| 0.0| 427|
+------------+------------+-----+

In [19]:
assembler = VectorAssembler(inputCols=["age","Pclass","gender_new","embarked_new"],outputCol="features")
X_assembler = assembler.transform(data)
X_assembler.show()

+----+------+----------+------------+------------+------------------+
 age|Pclass|gender_new|embarked_new|survived_new| features|
+----+------+----------+------------+------------+------------------+
22.0| 3| 0.0| 0.0| 0.0|[22.0,3.0,0.0,0.0]|
38.0| 1| 1.0| 1.0| 1.0|[38.0,1.0,1.0,1.0]|
26.0| 3| 1.0| 0.0| 1.0|[26.0,3.0,1.0,0.0]|
35.0| 1| 1.0| 0.0| 1.0|[35.0,1.0,1.0,0.0]|
35.0| 3| 0.0| 0.0| 0.0|[35.0,3.0,0.0,0.0]|
30.0| 3| 0.0| 2.0| 0.0|[30.0,3.0,0.0,2.0]|
54.0| 1| 0.0| 0.0| 0.0|[54.0,1.0,0.0,0.0]|
 2.0| 3| 0.0| 0.0| 0.0| [2.0,3.0,0.0,0.0]|
27.0| 3| 1.0| 0.0| 1.0|[27.0,3.0,1.0,0.0]|
14.0| 2| 1.0| 1.0| 1.0|[14.0,2.0,1.0,1.0]|
 4.0| 3| 1.0| 0.0| 1.0| [4.0,3.0,1.0,0.0]|
58.0| 1| 1.0| 0.0| 1.0|[58.0,1.0,1.0,0.0]|
20.0| 3| 0.0| 0.0| 0.0|[20.0,3.0,0.0,0.0]|
39.0| 3| 0.0| 0.0| 0.0|[39.0,3.0,0.0,0.0]|
14.0| 3| 1.0| 0.0| 0.0|[14.0,3.0,1.0,0.0]|
55.0| 2| 1.0| 0.0| 1.0|[55.0,2.0,1.0,0.0]|
 2.0| 3| 0.0| 2.0| 0.0| [2.0,3.0,0.0,2.0]|
30.0| 2| 0.0| 0.0| 1.0|[30.0,2.0,0.0,0.0]|
31.0| 3| 1.0| 0.0| 0.0|[31.0,3.0,1.0,0.0]|
30.0| 3| 1.0| 1.0| 1.0|[30.0,3.0,1.0,1.0]|
+----+------+----------+------------+------------+------------------+
only showing top 20 rows

In [20]:
X_train, X_test = X_assembler.randomSplit([0.8, 0.2])

In [21]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'survived_new')
rfModel = rf.fit(X_train)
predictions = rfModel.transform(X_test)
predictions.select("prediction", "survived_new", "features").show()

+----------+------------+------------------+
prediction|survived_new| features|
+----------+------------+------------------+
 1.0| 1.0| [1.0,2.0,0.0,1.0]|
 1.0| 0.0| [2.0,1.0,1.0,0.0]|
 1.0| 0.0| [2.0,3.0,1.0,0.0]|
 1.0| 1.0| [4.0,1.0,0.0,0.0]|
 0.0| 0.0| [4.0,3.0,0.0,0.0]|
 1.0| 1.0| [4.0,3.0,1.0,1.0]|
 1.0| 1.0| [5.0,3.0,1.0,1.0]|
 0.0| 1.0| [6.0,3.0,0.0,0.0]|
 1.0| 1.0| [7.0,2.0,1.0,0.0]|
 0.0| 1.0| [9.0,3.0,0.0,0.0]|
 0.0| 1.0|[11.0,1.0,0.0,0.0]|
 0.0| 1.0|[12.0,3.0,0.0,1.0]|
 1.0| 1.0|[13.0,3.0,1.0,1.0]|
 1.0| 1.0|[14.0,1.0,1.0,0.0]|
 1.0| 1.0|[14.0,3.0,1.0,1.0]|
 1.0| 1.0|[15.0,3.0,1.0,2.0]|
 1.0| 1.0|[16.0,1.0,1.0,0.0]|
 0.0| 0.0|[16.0,2.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
+----------+------------+------------------+
only showing top 20 rows

In [22]:
predictions.count()

Out[54]: 191

In [23]:
evaluator = MulticlassClassificationEvaluator(labelCol="survived_new", predictionCol="prediction", metricName="accuracy")
print("Accuracy : " + str(evaluator.evaluate(predictions)))

Accuracy : 0.8115183246073299

In [24]:
display(predictions)

age,Pclass,gender_new,embarked_new,survived_new,features,rawPrediction,probability,prediction
1.0,2,0.0,1.0,1.0,"List(1, 4, List(), List(1.0, 2.0, 0.0, 1.0))","List(1, 2, List(), List(3.586914174380941, 16.41308582561906))","List(1, 2, List(), List(0.17934570871904704, 0.820654291280953))",1.0
2.0,1,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 1.0, 1.0, 0.0))","List(1, 2, List(), List(1.8682934147907233, 18.131706585209276))","List(1, 2, List(), List(0.09341467073953616, 0.9065853292604638))",1.0
2.0,3,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 3.0, 1.0, 0.0))","List(1, 2, List(), List(9.584954616343209, 10.415045383656791))","List(1, 2, List(), List(0.47924773081716043, 0.5207522691828396))",1.0
4.0,1,0.0,0.0,1.0,"List(1, 4, List(), List(4.0, 1.0, 0.0, 0.0))","List(1, 2, List(), List(3.4381749190904434, 16.561825080909557))","List(1, 2, List(), List(0.17190874595452216, 0.8280912540454779))",1.0
4.0,3,0.0,0.0,0.0,"List(1, 4, List(), List(4.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
4.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(4.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
5.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(5.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
6.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(6.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
7.0,2,1.0,0.0,1.0,"List(1, 4, List(), List(7.0, 2.0, 1.0, 0.0))","List(1, 2, List(), List(1.9016267481240567, 18.09837325187594))","List(1, 2, List(), List(0.09508133740620285, 0.9049186625937972))",1.0
9.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(9.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(16.590359410857886, 3.409640589142113))","List(1, 2, List(), List(0.8295179705428943, 0.17048202945710564))",0.0


In [25]:
display(predictions)

age,Pclass,gender_new,embarked_new,survived_new,features,rawPrediction,probability,prediction
1.0,2,0.0,1.0,1.0,"List(1, 4, List(), List(1.0, 2.0, 0.0, 1.0))","List(1, 2, List(), List(3.586914174380941, 16.41308582561906))","List(1, 2, List(), List(0.17934570871904704, 0.820654291280953))",1.0
2.0,1,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 1.0, 1.0, 0.0))","List(1, 2, List(), List(1.8682934147907233, 18.131706585209276))","List(1, 2, List(), List(0.09341467073953616, 0.9065853292604638))",1.0
2.0,3,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 3.0, 1.0, 0.0))","List(1, 2, List(), List(9.584954616343209, 10.415045383656791))","List(1, 2, List(), List(0.47924773081716043, 0.5207522691828396))",1.0
4.0,1,0.0,0.0,1.0,"List(1, 4, List(), List(4.0, 1.0, 0.0, 0.0))","List(1, 2, List(), List(3.4381749190904434, 16.561825080909557))","List(1, 2, List(), List(0.17190874595452216, 0.8280912540454779))",1.0
4.0,3,0.0,0.0,0.0,"List(1, 4, List(), List(4.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
4.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(4.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
5.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(5.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
6.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(6.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
7.0,2,1.0,0.0,1.0,"List(1, 4, List(), List(7.0, 2.0, 1.0, 0.0))","List(1, 2, List(), List(1.9016267481240567, 18.09837325187594))","List(1, 2, List(), List(0.09508133740620285, 0.9049186625937972))",1.0
9.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(9.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(16.590359410857886, 3.409640589142113))","List(1, 2, List(), List(0.8295179705428943, 0.17048202945710564))",0.0


In [26]:
display(predictions)

age,Pclass,gender_new,embarked_new,survived_new,features,rawPrediction,probability,prediction
1.0,2,0.0,1.0,1.0,"List(1, 4, List(), List(1.0, 2.0, 0.0, 1.0))","List(1, 2, List(), List(3.586914174380941, 16.41308582561906))","List(1, 2, List(), List(0.17934570871904704, 0.820654291280953))",1.0
2.0,1,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 1.0, 1.0, 0.0))","List(1, 2, List(), List(1.8682934147907233, 18.131706585209276))","List(1, 2, List(), List(0.09341467073953616, 0.9065853292604638))",1.0
2.0,3,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 3.0, 1.0, 0.0))","List(1, 2, List(), List(9.584954616343209, 10.415045383656791))","List(1, 2, List(), List(0.47924773081716043, 0.5207522691828396))",1.0
4.0,1,0.0,0.0,1.0,"List(1, 4, List(), List(4.0, 1.0, 0.0, 0.0))","List(1, 2, List(), List(3.4381749190904434, 16.561825080909557))","List(1, 2, List(), List(0.17190874595452216, 0.8280912540454779))",1.0
4.0,3,0.0,0.0,0.0,"List(1, 4, List(), List(4.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
4.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(4.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
5.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(5.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
6.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(6.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
7.0,2,1.0,0.0,1.0,"List(1, 4, List(), List(7.0, 2.0, 1.0, 0.0))","List(1, 2, List(), List(1.9016267481240567, 18.09837325187594))","List(1, 2, List(), List(0.09508133740620285, 0.9049186625937972))",1.0
9.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(9.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(16.590359410857886, 3.409640589142113))","List(1, 2, List(), List(0.8295179705428943, 0.17048202945710564))",0.0


In [27]:
display(predictions)

age,Pclass,gender_new,embarked_new,survived_new,features,rawPrediction,probability,prediction
1.0,2,0.0,1.0,1.0,"List(1, 4, List(), List(1.0, 2.0, 0.0, 1.0))","List(1, 2, List(), List(3.586914174380941, 16.41308582561906))","List(1, 2, List(), List(0.17934570871904704, 0.820654291280953))",1.0
2.0,1,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 1.0, 1.0, 0.0))","List(1, 2, List(), List(1.8682934147907233, 18.131706585209276))","List(1, 2, List(), List(0.09341467073953616, 0.9065853292604638))",1.0
2.0,3,1.0,0.0,0.0,"List(1, 4, List(), List(2.0, 3.0, 1.0, 0.0))","List(1, 2, List(), List(9.584954616343209, 10.415045383656791))","List(1, 2, List(), List(0.47924773081716043, 0.5207522691828396))",1.0
4.0,1,0.0,0.0,1.0,"List(1, 4, List(), List(4.0, 1.0, 0.0, 0.0))","List(1, 2, List(), List(3.4381749190904434, 16.561825080909557))","List(1, 2, List(), List(0.17190874595452216, 0.8280912540454779))",1.0
4.0,3,0.0,0.0,0.0,"List(1, 4, List(), List(4.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
4.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(4.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
5.0,3,1.0,1.0,1.0,"List(1, 4, List(), List(5.0, 3.0, 1.0, 1.0))","List(1, 2, List(), List(5.688561440890413, 14.31143855910959))","List(1, 2, List(), List(0.2844280720445206, 0.7155719279554793))",1.0
6.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(6.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(10.158222140745652, 9.841777859254348))","List(1, 2, List(), List(0.5079111070372826, 0.4920888929627174))",0.0
7.0,2,1.0,0.0,1.0,"List(1, 4, List(), List(7.0, 2.0, 1.0, 0.0))","List(1, 2, List(), List(1.9016267481240567, 18.09837325187594))","List(1, 2, List(), List(0.09508133740620285, 0.9049186625937972))",1.0
9.0,3,0.0,0.0,1.0,"List(1, 4, List(), List(9.0, 3.0, 0.0, 0.0))","List(1, 2, List(), List(16.590359410857886, 3.409640589142113))","List(1, 2, List(), List(0.8295179705428943, 0.17048202945710564))",0.0


In [28]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="survived_new", featuresCol="features",maxIter=10)
gbt_model = gbt.fit(X_train)
gbt_predictions = gbt_model.transform(X_test)
gbt_predictions.select("prediction", "survived_new", "features").show()


+----------+------------+------------------+
prediction|survived_new| features|
+----------+------------+------------------+
 1.0| 1.0| [1.0,2.0,0.0,1.0]|
 1.0| 0.0| [2.0,1.0,1.0,0.0]|
 1.0| 0.0| [2.0,3.0,1.0,0.0]|
 1.0| 1.0| [4.0,1.0,0.0,0.0]|
 0.0| 0.0| [4.0,3.0,0.0,0.0]|
 1.0| 1.0| [4.0,3.0,1.0,1.0]|
 1.0| 1.0| [5.0,3.0,1.0,1.0]|
 0.0| 1.0| [6.0,3.0,0.0,0.0]|
 1.0| 1.0| [7.0,2.0,1.0,0.0]|
 0.0| 1.0| [9.0,3.0,0.0,0.0]|
 1.0| 1.0|[11.0,1.0,0.0,0.0]|
 0.0| 1.0|[12.0,3.0,0.0,1.0]|
 0.0| 1.0|[13.0,3.0,1.0,1.0]|
 1.0| 1.0|[14.0,1.0,1.0,0.0]|
 0.0| 1.0|[14.0,3.0,1.0,1.0]|
 1.0| 1.0|[15.0,3.0,1.0,2.0]|
 1.0| 1.0|[16.0,1.0,1.0,0.0]|
 0.0| 0.0|[16.0,2.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
+----------+------------+------------------+
only showing top 20 rows

In [29]:
evaluator1 = MulticlassClassificationEvaluator(labelCol="survived_new", predictionCol="prediction", metricName="accuracy")
print("Accuracy : " + str(evaluator1.evaluate(gbt_predictions)))

Accuracy : 0.8010471204188482

In [30]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(labelCol="survived_new", featuresCol="features")
nb_model = nb.fit(X_train)
nb_predictions = nb_model.transform(X_test)
nb_predictions.select("prediction", "survived_new", "features").show()


+----------+------------+------------------+
prediction|survived_new| features|
+----------+------------+------------------+
 0.0| 1.0| [1.0,2.0,0.0,1.0]|
 1.0| 0.0| [2.0,1.0,1.0,0.0]|
 1.0| 0.0| [2.0,3.0,1.0,0.0]|
 0.0| 1.0| [4.0,1.0,0.0,0.0]|
 0.0| 0.0| [4.0,3.0,0.0,0.0]|
 1.0| 1.0| [4.0,3.0,1.0,1.0]|
 1.0| 1.0| [5.0,3.0,1.0,1.0]|
 0.0| 1.0| [6.0,3.0,0.0,0.0]|
 1.0| 1.0| [7.0,2.0,1.0,0.0]|
 0.0| 1.0| [9.0,3.0,0.0,0.0]|
 0.0| 1.0|[11.0,1.0,0.0,0.0]|
 0.0| 1.0|[12.0,3.0,0.0,1.0]|
 1.0| 1.0|[13.0,3.0,1.0,1.0]|
 1.0| 1.0|[14.0,1.0,1.0,0.0]|
 1.0| 1.0|[14.0,3.0,1.0,1.0]|
 1.0| 1.0|[15.0,3.0,1.0,2.0]|
 1.0| 1.0|[16.0,1.0,1.0,0.0]|
 0.0| 0.0|[16.0,2.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
+----------+------------+------------------+
only showing top 20 rows

In [31]:
evaluator2 = MulticlassClassificationEvaluator(labelCol="survived_new", predictionCol="prediction", metricName="accuracy")
print("Accuracy : " + str(evaluator2.evaluate(nb_predictions)))

Accuracy : 0.7958115183246073

In [32]:
from pyspark.ml.classification import LinearSVC
svm = LinearSVC(maxIter = 10 ,labelCol="survived_new", featuresCol="features")
svm_model = nb.fit(X_train)
svm_predictions = nb_model.transform(X_test)
svm_predictions.select("prediction", "survived_new", "features").show()

+----------+------------+------------------+
prediction|survived_new| features|
+----------+------------+------------------+
 0.0| 1.0| [1.0,2.0,0.0,1.0]|
 1.0| 0.0| [2.0,1.0,1.0,0.0]|
 1.0| 0.0| [2.0,3.0,1.0,0.0]|
 0.0| 1.0| [4.0,1.0,0.0,0.0]|
 0.0| 0.0| [4.0,3.0,0.0,0.0]|
 1.0| 1.0| [4.0,3.0,1.0,1.0]|
 1.0| 1.0| [5.0,3.0,1.0,1.0]|
 0.0| 1.0| [6.0,3.0,0.0,0.0]|
 1.0| 1.0| [7.0,2.0,1.0,0.0]|
 0.0| 1.0| [9.0,3.0,0.0,0.0]|
 0.0| 1.0|[11.0,1.0,0.0,0.0]|
 0.0| 1.0|[12.0,3.0,0.0,1.0]|
 1.0| 1.0|[13.0,3.0,1.0,1.0]|
 1.0| 1.0|[14.0,1.0,1.0,0.0]|
 1.0| 1.0|[14.0,3.0,1.0,1.0]|
 1.0| 1.0|[15.0,3.0,1.0,2.0]|
 1.0| 1.0|[16.0,1.0,1.0,0.0]|
 0.0| 0.0|[16.0,2.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
 0.0| 0.0|[16.0,3.0,0.0,0.0]|
+----------+------------+------------------+
only showing top 20 rows

In [33]:
evaluator3 = MulticlassClassificationEvaluator(labelCol="survived_new", predictionCol="prediction", metricName="accuracy")
print("Accuracy : " + str(evaluator3.evaluate(svm_predictions)))

Accuracy : 0.7958115183246073